![Header](imgs/santander_custsat_red.png)

### Classificação de usuários felizes ou não

O desafio [Santander Customer Satisfaction](https://www.kaggle.com/c/santander-customer-satisfaction) do [Kaggle](https://www.kaggle.com/) é uma desafio de classificação binária que tem como finalidade prever se os clientes estão gostando ou não dos serviços serviços oferecidos pelo Santander.

Iremos realizar a classificação utilizando uma KNN e Regressão Lasso

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import dos dados

df = pd.read_csv('train_santander.csv')

In [3]:
df.shape

(76020, 371)

### Divisão do Dataset

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df[df.columns.drop('TARGET')], df['TARGET'], test_size=0.3, random_state=42)

### KNN

In [6]:
from sklearn.neighbors import KNeighborsClassifier

In [7]:
neigh = KNeighborsClassifier(n_neighbors=3)

In [8]:
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [9]:
y_pred = neigh.predict(X_test)

### Métrica

Iremos utilizar o [Curva ROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) assimo como é utilizado na competição

In [10]:
from sklearn.metrics import roc_auc_score

In [11]:
roc_auc_score(y_test, y_pred)

0.5020875410285385

### Standardização

Vamos standizar os dados para tentar melhorar as predições

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [14]:
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

In [15]:
neigh_std = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train_std, y_train)

KNeighborsClassifier(n_neighbors=3)

In [16]:
y_pred_std = neigh.predict(X_test)
roc_auc_score(y_test, y_pred_std)

0.5

### Grid Search

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
params = {
    'n_neighbors': [2, 3, 5, 7, 10],
    #'splitter'  : ['best', 'random'],
    #'min_samples_split': [2,5],
    #'min_samples_leaf': [1,3]
}

# nota: não consegui rodar com mais parâmetros pq minha máquina não terminava
#       o treinamento

In [19]:
neigh_gs = KNeighborsClassifier()

In [20]:
clf = GridSearchCV(neigh_gs, params)

In [21]:
clf.fit(X_test, y_test)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [2, 3, 5, 7, 10]})

In [22]:
clf.best_params_

{'n_neighbors': 10}

In [23]:
clf.best_estimator_.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [24]:
roc_auc_score(y_train, clf.best_estimator_.predict(X_train))

0.5

In [25]:
roc_auc_score(y_test, clf.best_estimator_.predict(X_test))

0.5

### Lasso (Least Absolute Selection and Shrinkage Operator)

Lasso é uma regressão que utiliza de regularização para que o modelo não fique muito adaptado ao dataset de treinamento. A regressão Lasso utliza o parâmetro alpha para realizar cálculos para "penalizar" os pesos da regressão para que eles não fiquem "overfitados".

In [26]:
from sklearn import linear_model

In [27]:
reg = linear_model.Lasso(alpha=0.1)

In [28]:
reg.fit(X_train, y_train)

/home/iara/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 983.4890173271623, tolerance: 0.20106786372007243
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)

In [29]:
y_pred = reg.predict(X_test)

In [30]:
roc_auc_score(y_test, y_pred)

0.7419029653377026

In [31]:
reg_std = linear_model.Lasso(alpha=0.1)
reg_std.fit(X_train_std, y_train)

Lasso(alpha=0.1)

In [32]:
y_pred_std = reg_std.predict(X_test_std)

In [33]:
roc_auc_score(y_test, y_pred_std)

0.5

In [34]:
## por algum motivo os resultados utilizando standardização dos dados foram piores